<a href="https://colab.research.google.com/github/YBenPan/oil-forecast/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import linear_model

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import datetime
%matplotlib inline

In [7]:
!rm -rf *
#!ls -a
!git clone https://github.com/YBenPan/oil-forecast.git 
#!ls oil-forecast/data

'rm' is not recognized as an internal or external command,
operable program or batch file.
'git' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
df = pd.read_csv("oil-forecast/Oil3Data.csv", sep = ",", index_col = 'Date', parse_dates = True)
df = df.dropna(axis = 0)

FileNotFoundError: [Errno 2] No such file or directory: 'oil-forecast/Oil3Data.csv'

In [ ]:
df = df[1000:]
df

Build ARIMA model: 

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
from dateutil.parser import parse

additive_decomposition = seasonal_decompose(df['Stock'], model = 'additive', freq = 30)
multiplicative_decomposition = seasonal_decompose(df['Stock'], model = 'multiplicative', freq = 30)

plt.rcParams.update({'figure.figsize': (16,12)})
multiplicative_decomposition.plot().suptitle('Multiplicative Decomposition Using Data From 2009 to 2010')
additive_decomposition.plot().suptitle('Additive Decomposition Using Data From 2009 to 2020')
plt.show()

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(df['Stock'])

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
model = ARIMA(df['Stock'], order = (2, 1, 2), freq = 'W-FRI')
model_fit = model.fit()
print(model_fit.summary())

In [ ]:
model_fit.plot_predict(dynamic = False, start = '2017-01-06')
plt.show()

In [ ]:
residuals = pd.DataFrame(model_fit.resid)
residuals.plot()

Model Validation:

In [ ]:
dflen = len(df)
testlen = round(dflen / 5)
trainlen = dflen - testlen
train = df[0:trainlen]
test = df[trainlen: dflen]
#train
#test

In [ ]:
model = ARIMA(df['Stock'], order = (1, 2, 2))
model_fit = model.fit()
output = model_fit.forecast()
#output[0][0]

In [ ]:
def build(train, test):
    history = train
    predictions = pd.DataFrame()
    for i in range(len(test)):
        model = ARIMA(history['Stock'], order = (1, 1, 2), freq = 'W-FRI')
        model_fit = model.fit()
        output = model_fit.forecast()
        predicted = output[0][0]
        added_obj = test.iloc[i]
        predicted_series = pd.Series([predicted])
        added_obj = added_obj.append(predicted_series) 
        added_obj.index = ['Stock', 'Stock_change', 'Price', 'Price_change', 'Days of Supply', 'DoS_change', 'Predicted_Stock']
        added_obj = added_obj.rename(i)
        predictions = predictions.append(added_obj)
        history = history.append(test.iloc[i])
    return predictions

In [ ]:
res = build(train, test)
res = res[['Stock', 'Stock_change', 'Predicted_Stock', 'Price', 'Price_change', 'Days of Supply', 'DoS_change']]

In [ ]:
pre1 = res['Stock']
pre2 = res['Predicted_Stock']
pre3 = res.index
plt.title("Actual (Blue) vs Predicted (Red)")
plt.plot(pre3, pre1, label = "Actual", color = "Blue")
plt.plot(pre3, pre2, label = "Predicted", color = "Red")

In [ ]:
print("Root Mean Squared Error:", math.sqrt(metrics.mean_squared_error(res['Predicted_Stock'], res['Stock'])))

In [ ]:
x_train = df.loc[:, 'Stock'].values
#print(x_train[:15])
#print(type(x_train))
y_train = df.loc[:, 'Price'].values
#print(y_train)
#x_test = res.loc[:, 'Predicted_Stock'].values
x_test = res.loc[:, 'Predicted_Stock'].values
#print(x_test)
y_test = res.loc[:, 'Price'].values
#print(y_test)

In [ ]:
#x_train = x_train.reshape(-1, 1)
#x_test = x_test.reshape(-1, 1)

Polynomial Regression model

In [ ]:
#regressor = LinearRegression()
regressor = np.poly1d(np.polyfit(x_train, y_train, 3))
#print(regressor.coef_)
#print(regressor.intercept_)

In [ ]:
plt.scatter(x_train, y_train, color = "red")
plt.plot(x_train, regressor(x_train), color = "blue")
plt.title("Stock vs Price")
plt.xlabel("Stock")
plt.ylabel("Price")

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(y_train, regressor(x_train)))

In [ ]:
predicted = regressor(x_test)
res['Predicted_Price'] = predicted
#res
#print(type(predicted))

In [ ]:
print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, predicted))
print("Mean Squared Error:", metrics.mean_squared_error(y_test, predicted))
print("Root Mean Squared Error:", math.sqrt(metrics.mean_squared_error(y_test, predicted)))

In [ ]:
res.plot(y = ["Price", "Predicted_Price"])

Model Evaluation. Predicts the Price

In [ ]:
model = ARIMA(df['Stock'], order = (1, 2, 2))
model_fit = model.fit()
output = model_fit.forecast()
newStock = output[0][0]
newPrice = regressor(newStock)
print("The next price will be:", newPrice)